# Machine learning for solid solutions (Li2TiS3)

This notebook is divided into these sections:
- [LTS dataset](#lts)
    - write CRYSTAL input files
    - read CRYSTAL output files
- [Descriptors](#descriptors)
    - [Coulomb Matrix (CM)](#CM)
    - Smooth Overlap of Atomic Positions (SOAP)
    - Many Body Tensor Representation (MBTR)
    - Ewald Sum Matrix
    - Sine Matrix
    
- [Machine learning](#ml)
    - linear regression
- [Protocol](#protocol)
    - [simmetry analysis](#symmetry)

In [1]:
import os
import copy
import json
import itertools
import shutil as sh
from pathlib import Path
import numpy as np
import pandas as pd
from datetime import datetime

from CRYSTALpytools.crystal_io import Crystal_output, Crystal_input, Crystal_density, Crystal_gui
from CRYSTALpytools.convert import cry_gui2pmg, cry_out2pmg
from CRYSTALpytools.utils import view_pmg

from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.io.cif import CifWriter
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer, PointGroupAnalyzer

from ase.visualize import view

#from dscribe.descriptors import CoulombMatrix

from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, max_error
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,15)

# <a id='lts'>LTS dataset - pymatgen</a>

In [2]:
# New atom
new_atom = 'Li'

# Read the confcount output
cry_output = Crystal_output().read_cry_output('data/crystal/lts/lts_confcount.out')
cry_output.get_config_analysis()

# Read the initial structure (before substitution)
original_structure_gui =  Crystal_gui().read_cry_gui('data/crystal/lts/lts_confcount.gui')
original_structure = cry_gui2pmg(original_structure_gui)

structures_lts = []
li_atoms = []
ti_atoms = []
for j,substitutions in enumerate(cry_output.atom_type2):
    new_structure = original_structure.copy()
    for i in substitutions:
        new_structure.replace(i-1,new_atom)
    structures_lts.append(new_structure)
    ti_atoms.append((np.array(cry_output.atom_type1[j])-1).tolist())
    li_atoms.append((np.array(cry_output.atom_type2[j])-1).tolist())

structures = copy.deepcopy(structures_lts)

## Read the energy and band gap

#### Single point

In [47]:
energies_sp = []
gap_sp = []
for i in range(len(structures)):
    crystal_output = Crystal_output().read_cry_output('./data/crystal/lts/sp/output/lts_sp_%s.out'%str(i))
    if crystal_output.get_final_energy() != None:
        energies_sp.append(crystal_output.get_final_energy())
        gap_sp.append(crystal_output.get_band_gap())

EXITING: a .out file needs to be specified


AssertionError: 

#### Optimised geometry

In [4]:
energies_opt = []
gap_opt = []
for i in range(len(structures)):
    #crystal_output = Crystal_output().read_cry_output('./data/crystal/lts/sp/output/lts_sp_%s.out'%str(i))
    if crystal_output.get_final_energy() != None:
        energies_opt.append(crystal_output.get_final_energy())
        gap_opt.append(crystal_output.get_band_gap())

## Covert the LTS dataset to ASE

In [3]:
ase_structures = []
for i in range(len(structures)):
    ase_struct = AseAtomsAdaptor().get_atoms(structures[i])
    ase_structures.append(ase_struct)

# <a id='descriptors'>Descriptors</a>

## <a id='CM'>Coulomb Matrix</a>

$M_{ij}^{Coulomb}= \Bigl\{^{0.5Z_i^{2.4} \; \; for \; i=j}_{\frac{Z_iZ_j}{R_{ij}}\;\;\;\; for \; i\neq j}$

(taken from the Dscribe website)

The diagonal elements are the interaction of an atom with itself and are a polynomial fit of the atomic energies to the nuclear charge $Z_i$. 

The off-diagonal elements represent the Coulomb repulsion between nuclei $i$ and $j$.

In [4]:
#dscribe descriptors CM
from dscribe.descriptors import CoulombMatrix

cm_dscribe = []
cm_ds = CoulombMatrix(n_atoms_max=54,permutation="eigenspectrum")
start = datetime.now()
for i,ase_struct in enumerate(ase_structures):
    dscribe_matrix = cm_ds.create(ase_struct)
    cm_dscribe.append(dscribe_matrix)
    now = datetime.now()
    if (i+1)%200 == 0: 
        print("matrices read:", len(cm_dscribe),", time:", (now - start))
cm_dscribe = np.array(cm_dscribe)   
print('Number of matrices read: ', len(cm_dscribe))
print("--- %s time taken ---" % (datetime.now()- start))

matrices read: 200 , time: 0:00:00.025629
matrices read: 400 , time: 0:00:00.056160
matrices read: 600 , time: 0:00:00.082902
matrices read: 800 , time: 0:00:00.108168
matrices read: 1000 , time: 0:00:00.133687
matrices read: 1200 , time: 0:00:00.159049
matrices read: 1400 , time: 0:00:00.184340
matrices read: 1600 , time: 0:00:00.209612
matrices read: 1800 , time: 0:00:00.234741
matrices read: 2000 , time: 0:00:00.260694
matrices read: 2200 , time: 0:00:00.287076
matrices read: 2400 , time: 0:00:00.313768
matrices read: 2600 , time: 0:00:00.343454
matrices read: 2800 , time: 0:00:00.369705
matrices read: 3000 , time: 0:00:00.395439
matrices read: 3200 , time: 0:00:00.421684
matrices read: 3400 , time: 0:00:00.447402
matrices read: 3600 , time: 0:00:00.473046
matrices read: 3800 , time: 0:00:00.498897
matrices read: 4000 , time: 0:00:00.524510
Number of matrices read:  4023
--- 0:00:00.530027 time taken ---


### Inspect the descriptor

#### Full matrix

In [80]:
cm_ds = CoulombMatrix(n_atoms_max=54,permutation='none',flatten=False)
print(cm_ds.create(ase_structures[0]))
print(cm_ds.create(ase_structures[0]).shape)

[[  6.98330508   2.50278676   2.50278676 ...  18.87719988   6.29239996
   10.89875643]
 [  2.50278676   6.98330508   1.25139338 ...  10.89875643   4.33072666
    6.29239996]
 [  2.50278676   1.25139338   6.98330508 ...  10.89875643  10.89875643
   18.87719988]
 ...
 [ 18.87719988  10.89875643  10.89875643 ... 388.02344103  35.59518946
   71.19037891]
 [  6.29239996   4.33072666  10.89875643 ...  35.59518946 388.02344103
   71.19037891]
 [ 10.89875643   6.29239996  18.87719988 ...  71.19037891  71.19037891
  388.02344103]]
(54, 54)


#### Eigenvalues only

In [81]:
cm_ds = CoulombMatrix(n_atoms_max=54,permutation='eigenspectrum')
print(cm_ds.create(ase_structures[0]))
print(cm_ds.create(ase_structures[0]).shape)

[2679.77619447 1133.67577892  913.07374493  841.08011951  712.6289803
  708.830714    692.25295309  656.98047438  655.93789699  611.12050477
  467.43255314  434.36093126  405.95650918  373.37691024  360.90885612
  360.73370676  330.59963438  323.17143181  321.91922861  319.70671666
  317.12026943  310.00661992  304.11487923  301.61723033  299.21583906
  298.12478161  296.60093965  292.83209275  292.52858515  292.14396424
  291.40954568  283.9316412   283.8511157   283.12161825  282.57751345
  280.79923438   12.8666636     7.39275912    7.03464999    5.57408163
    5.28041001    4.54978872    4.52989384    4.34655433    4.22686951
    4.14798147    3.80471049    3.74741204    3.72709043    3.52626779
    3.47022584    3.37089434    3.33626234    3.28758742]
(54,)


In [10]:
# Save cme to file
np.save('./data/descriptors/cm_dscribe.npy',cm_dscribe,allow_pickle=True)

In [21]:
# Read cme from file
cm_dscribe = np.load('./data/descriptors/cm_dscribe.npy',allow_pickle=True)
cm_dscribe.shape

(4023, 54)

### Test the parameters

#there's not really any parameters that needed to be set for CME as there is only the maximum atoms

# SOAP

$P_{nn'l}^{Z_1Z_2} = \pi \sqrt{\frac{8}{2l+1}}\sum c_{nlm}^{Z_1}*c_{n'lm}^{Z_2}$

where the $n$ indices for the different radial basis functions for up to $n_{max}$, $l$ is the angular degree of the spherical harmonics up to $l_{max}$. 
<br>
The defult for Dscribe descriptors are the spherical gaussian type orbitals as riadial baiss functions.

In [8]:
#setting up the SOAP descriptor

from dscribe.descriptors import SOAP


rcut = 6.0
nmax = 8
lmax = 6

soap = SOAP(
    species = ["Li", "Ti", "S"],
    periodic= True,
    r_cut=rcut,
    n_max=nmax,
    l_max=lmax
)

In [9]:
soap_dscribe = []
start = datetime.now()
for i,ase_struct in enumerate(ase_structures):
    soap_matrix = soap.create(ase_struct)
    #soap_matrix = np.real(soap_matrix)
    soap_dscribe.append(soap_matrix)
    now = datetime.now()
    if int(len(soap_dscribe)) == 20:
        print("matrices read:", len(soap_dscribe),", time:", (now - start))
    if int(len(soap_dscribe))%200 == 0: 
        print("matrices read:", len(soap_dscribe),", time:", (now - start))
    
print('Number of matrices read: ', len(soap_dscribe))
print("--- %s time taken ---" % ((datetime.now() - start)))

matrices read: 20 , time: 0:00:00.315297
matrices read: 200 , time: 0:00:02.740344
matrices read: 400 , time: 0:00:05.487684
matrices read: 600 , time: 0:00:08.235443
matrices read: 800 , time: 0:00:10.956856
matrices read: 1000 , time: 0:00:13.713330
matrices read: 1200 , time: 0:00:16.463137
matrices read: 1400 , time: 0:00:19.200754
matrices read: 1600 , time: 0:00:21.947930
matrices read: 1800 , time: 0:00:25.024074
matrices read: 2000 , time: 0:00:27.867552
matrices read: 2200 , time: 0:00:30.692632
matrices read: 2400 , time: 0:00:33.656431
matrices read: 2600 , time: 0:00:36.643515
matrices read: 2800 , time: 0:00:39.664166
matrices read: 3000 , time: 0:00:42.882371
matrices read: 3200 , time: 0:00:45.853716
matrices read: 3400 , time: 0:00:48.810812
matrices read: 3600 , time: 0:00:52.054134
matrices read: 3800 , time: 0:00:55.130138
matrices read: 4000 , time: 0:00:58.156553
Number of matrices read:  4023
--- 0:00:58.511120 time taken ---


In [18]:
# Save SOAP to file
np.save('./data/descriptors/soap_dscribe.npy',soap_dscribe,allow_pickle=True)

In [22]:
# Read SOAP from file
soap_dscribe = np.load('./data/descriptors/soap_dscribe.npy',allow_pickle=True)
soap_dscribe.shape

(4023, 54, 2100)

In [9]:
print(soap_dscribe[0])

[[ 1.31353093e-02  7.73465020e-02  1.38221146e-01 ...  1.96508259e-01
  -3.96163258e-01  9.21863184e-01]
 [ 1.31353093e-02  7.73465020e-02  1.38221146e-01 ...  1.96508259e-01
  -3.96163258e-01  9.21863184e-01]
 [ 1.31353093e-02  7.73465020e-02  1.38221146e-01 ...  1.96508259e-01
  -3.96163258e-01  9.21863184e-01]
 ...
 [ 1.11314369e-03  4.06866813e-04  1.98625315e-02 ...  4.60933402e-01
  -6.75916720e-01  9.92822626e-01]
 [ 1.11314369e-03  4.06866813e-04  1.98625315e-02 ...  4.60933402e-01
  -6.75916720e-01  9.92822626e-01]
 [ 1.11314369e-03  4.06866813e-04  1.98625315e-02 ...  4.60933402e-01
  -6.75916720e-01  9.92822626e-01]]


In [10]:
print(soap_dscribe[0].shape)

(54, 2100)


###  Inspection of the descriptor

In [45]:
#reference

rcut = 20
nmax = 8
lmax = 6
soap = SOAP(
            species = ["Li", "Ti", "S"],
            periodic= True,
            r_cut=rcut,
            n_max=nmax,
            l_max=lmax,

        )
        

soap_max = soap.create(ase_structures[0])

In [49]:
print(soap_max)
flat = soap_max.flatten()
print(flat.shape)


[[ 2.45363608e-01  1.45172131e+00 -3.82618049e-01 ...  1.20095615e+01
  -1.96018038e+01  3.19962695e+01]
 [ 2.45363608e-01  1.45172131e+00 -3.82618049e-01 ...  1.20095615e+01
  -1.96018038e+01  3.19962695e+01]
 [ 2.45363608e-01  1.45172131e+00 -3.82618049e-01 ...  1.20095615e+01
  -1.96018038e+01  3.19962695e+01]
 ...
 [ 2.03856634e-02  3.98716685e-02  2.20314643e-01 ...  2.23668637e+01
  -2.69527879e+01  3.24809737e+01]
 [ 2.03856634e-02  3.98716685e-02  2.20314643e-01 ...  2.23668637e+01
  -2.69527879e+01  3.24809737e+01]
 [ 2.03856634e-02  3.98716685e-02  2.20314643e-01 ...  2.23668637e+01
  -2.69527879e+01  3.24809737e+01]]
(113400,)


In [34]:
#changing the parameters to see how the value change:
#rcut = np.arange(6,21,1)
rcut = np.arange(6,13,1)
nmax = list(range(1,13))
lmax = np.arange(1,13,1)
dct = {}
#nmax = 8

#lmax = 6
start = datetime.now()
for r in rcut:
    for n in nmax:
        for l in lmax:
            soap = SOAP(
                species = ["Li", "Ti", "S"],
                periodic= True,
                    r_cut=r,
                    n_max=n,
                    l_max=l
                )
      
            dct['r_%s_n_%s_l_%s' % (r, n, l) ] = []
            for a in range(10):
                soap_matrix = soap.create(ase_structures[a])
#            soap_matrix = soap.create(ase_structures[1:10])
                dct['r_%s_n_%s_l_%s' % (r, n, l)].append(soap_matrix)
                now = datetime.now()
        
    print('rcut =',r , 'nmax =',n , 'lmax =',l, 'done')
    print('time taken to finish cycle with r=%s :%s' % (r, datetime.now() - start))        

rcut = 6 nmax = 12 lmax = 12 done
time taken to finish cycle with r=6 :0:00:35.787226
rcut = 7 nmax = 12 lmax = 12 done
time taken to finish cycle with r=7 :0:01:18.438452
rcut = 8 nmax = 12 lmax = 12 done
time taken to finish cycle with r=8 :0:02:21.214211
rcut = 9 nmax = 12 lmax = 12 done
time taken to finish cycle with r=9 :0:03:34.268824
rcut = 10 nmax = 12 lmax = 12 done
time taken to finish cycle with r=10 :0:05:12.565188
rcut = 11 nmax = 12 lmax = 12 done
time taken to finish cycle with r=11 :0:07:03.181612
rcut = 12 nmax = 12 lmax = 12 done
time taken to finish cycle with r=12 :0:09:24.924673


In [76]:
#lets assume ideal lmax and nmax = 12
#let's use linear regression and plot the MAE

#Testing Rcut values 
    #need to find the ideal n and l values as well
testE = np.arange(400,500,10)
para_test_soap = pd.DataFrame(testE)

#need to split all of the descriptors to training set and test set

for i in rcut:
    para_test_soap['r_%s_n_12_l_12' %str(i)] = dct['r_%s_n_12_l_12' %str(i)]
    X_train, X_test, y_train, y_test = train_test_split(para_test_soap['r_%s_n_12_l_12' %str(i)], para_test_soap[0], random_state=1, test_size = 0.5)
    scaler = MinMaxScaler()  
    #scaler.fit(X_train)  
    #X_train = scaler.transform(X_train)  
    #X_test = scaler.transform(X_test) 

#and then need to run to fit for each r values
#predictions for the energies for each r values
#calculations for the errors
#plotting of the errors against the increasing rcut value

## MBTR Descriptor

In [60]:
from dscribe.descriptors import MBTR

#setting up the MBTR descriptor
mbtr = MBTR(
    species=["Li", "Ti", "S"],
    k1={
        "geometry": {"function": "atomic_number"},
        "grid": {"min": 0, "max": 8, "n": 200, "sigma": 0.1},
    },
    k2={
        "geometry": {"function": "inverse_distance"},
        "grid": {"min": 0, "max": 1, "n": 100, "sigma": 0.1},
        "weighting": {"function": "exp", "scale": 0.5, "threshold": 1e-3},
    },
    k3={
        "geometry": {"function": "cosine"},
        "grid": {"min": -1, "max": 1, "n": 100, "sigma": 0.1},
        "weighting": {"function": "exp", "scale": 0.5, "threshold": 1e-3},
    },
    periodic=True,
    normalization="l2_each",
    flatten=True,
    #sparse=False (only changes the return type)
)

In [90]:
#MBTR descriptor
mbtr_dscribe = []
start = datetime.now()
for i,ase_struct in enumerate(ase_structures):
    mbtr_matrix = mbtr.create(ase_struct)
    #mbtr_matrix = np.real(mbtr_matrix)
    mbtr_dscribe.append(mbtr_matrix)
    now = datetime.now()
    if int(len(mbtr_dscribe)) == 20:
        print("matrices read:", len(mbtr_dscribe),", time:", (now - start))
    if int(len(mbtr_dscribe))%200 == 0: 
        print("matrices read:", len(mbtr_dscribe),", time:", (now - start))
print('Number of matrices read: ', len(mbtr_dscribe))
print("--- %s time taken ---" % ((datetime.now() - start)))

matrices read: 20 , time: 0:00:18.676465
matrices read: 200 , time: 0:03:04.618426
matrices read: 400 , time: 0:06:11.888169
matrices read: 600 , time: 0:09:21.978380
matrices read: 800 , time: 0:12:30.620042
matrices read: 1000 , time: 0:15:43.693479
matrices read: 1200 , time: 0:18:55.219530
matrices read: 1400 , time: 0:22:06.129798
matrices read: 1600 , time: 0:25:17.181301
matrices read: 1800 , time: 0:28:27.808202
matrices read: 2000 , time: 0:31:38.854499
matrices read: 2200 , time: 0:34:50.827510
matrices read: 2400 , time: 0:38:02.551797
matrices read: 2600 , time: 0:41:15.546765
matrices read: 2800 , time: 0:44:27.683920
matrices read: 3000 , time: 0:47:43.159158
matrices read: 3200 , time: 0:50:58.429130
matrices read: 3400 , time: 0:54:15.664723
matrices read: 3600 , time: 0:57:28.496517
matrices read: 3800 , time: 1:00:42.225574
matrices read: 4000 , time: 1:03:56.288877
Number of matrices read:  4023
--- 1:04:18.460093 time taken ---


In [ ]:
# Save MBTR to file
np.save('./data/descriptors/MBTR_dscribe.npy',mbtr_dscribe,allow_pickle=True)

In [ ]:
# Read MBTR from file
MBTR_dscribe = np.load('./data/descriptors/MBTR_dscribe.npy',allow_pickle=True)
MBTR_dscribe.shape

In [61]:
mbtr_1 = mbtr.create(ase_structures[0])

In [65]:
print(mbtr_1)
print(np.shape(mbtr_1))

[0.         0.         0.         ... 0.00166096 0.00134801 0.00105044]
(3000,)


### Inspection of the descriptor

## Local Many Body Tensor Representation

In [109]:
from dscribe.descriptors import LMBTR
lmbtr = LMBTR(
    species=["Li", "Ti", "S"],
k2={
        "geometry": {"function": "inverse_distance"},
        "grid": {"min": 0, "max": 1, "n": 100, "sigma": 0.1},
        "weighting": {"function": "exp", "scale": 0.5, "threshold": 1e-3},
    },
    k3={
        "geometry": {"function": "cosine"},
        "grid": {"min": -1, "max": 1, "n": 100, "sigma": 0.1},
        "weighting": {"function": "exp", "scale": 0.5, "threshold": 1e-3},
    },
    periodic=True,
    normalization="l2_each",
    flatten =True
)


In [110]:
lmbtr_dscribe = []
start = datetime.now()
for i,ase_struct in enumerate(ase_structures):
    lmbtr_matrix = lmbtr.create(ase_struct)
    lmbtr_dscribe.append(lmbtr_matrix)
    now = datetime.now()
    if int(len(lmbtr_dscribe)) == 20:
        print("matrices read:", len(lmbtr_dscribe),", time:", (now - start))
    if int(len(lmbtr_dscribe))%200 == 0: 
        print("matrices read:", len(lmbtr_dscribe),", time:", (now - start))
print('Number of matrices read: ', len(lmbtr_dscribe))
print("--- %s time taken ---" % ((datetime.now() - start)))

matrices read: 20 , time: 0:00:29.723770


KeyboardInterrupt: 

In [111]:
print(lmbtr_dscribe[0])
print(lmbtr_dscribe[0].shape)


[[0.         0.         0.         ... 0.00024525 0.00019345 0.00014758]
 [0.         0.         0.         ... 0.00024525 0.00019345 0.00014758]
 [0.         0.         0.         ... 0.00024525 0.00019345 0.00014758]
 ...
 [0.         0.         0.         ... 0.00168127 0.00142138 0.00118152]
 [0.         0.         0.         ... 0.00168127 0.00142138 0.00118152]
 [0.         0.         0.         ... 0.00168127 0.00142138 0.00118152]]
(54, 2600)


## Ewald Sum

In [83]:
from dscribe.descriptors import EwaldSumMatrix

In [84]:
ewald = EwaldSumMatrix(
    n_atoms_max=54,
)

In [85]:
ewald_dscribe = []
start = datetime.now()
for i,ase_struct in enumerate(ase_structures):
    ewald_matrix = ewald.create(ase_struct)
    ewald_dscribe.append(ewald_matrix)
    now = datetime.now()
    if int(len(ewald_dscribe)) == 20:
        print("matrices read:", len(ewald_dscribe),", time:", (now - start))
    if int(len(ewald_dscribe))%200 == 0: 
        print("matrices read:", len(ewald_dscribe),", time:", (now - start))
print('Number of matrices read: ', len(ewald_dscribe))
print("--- %s time taken ---" % ((datetime.now() - start)))

matrices read: 20 , time: 0:00:00.762346
matrices read: 200 , time: 0:00:06.792230
matrices read: 400 , time: 0:00:13.579437
matrices read: 600 , time: 0:00:20.349381
matrices read: 800 , time: 0:00:27.079578
matrices read: 1000 , time: 0:00:33.808773
matrices read: 1200 , time: 0:00:40.493322
matrices read: 1400 , time: 0:00:47.260249
matrices read: 1600 , time: 0:00:53.943471
matrices read: 1800 , time: 0:01:00.670301
matrices read: 2000 , time: 0:01:07.571398
matrices read: 2200 , time: 0:01:14.584890
matrices read: 2400 , time: 0:01:21.751078
matrices read: 2600 , time: 0:01:29.030560
matrices read: 2800 , time: 0:01:36.374011
matrices read: 3000 , time: 0:01:43.723998
matrices read: 3200 , time: 0:01:51.355262
matrices read: 3400 , time: 0:01:58.785667
matrices read: 3600 , time: 0:02:06.375701
matrices read: 3800 , time: 0:02:13.918714
matrices read: 4000 , time: 0:02:21.487620
Number of matrices read:  4023
--- 0:02:22.392510 time taken ---


In [88]:
#one of the ewald sum matrix descriptor
print(ewald_dscribe[0])
print(ewald_dscribe[0].shape)

[-72.72593755   4.34200165   4.34200165 ...  -0.4560209    0.0807397
  -1.35234181]
(2916,)


## Sine Matrix

In [90]:
from dscribe.descriptors import SineMatrix

# Setting up the sine matrix descriptor
sine = SineMatrix(
    n_atoms_max=54,
    permutation="eigenspectrum",
    sparse=False,
    #flatten=True
)

In [91]:
sine_dscribe = []
start = datetime.now()
for i,ase_struct in enumerate(ase_structures):
    sine_matrix = sine.create(ase_struct)
    sine_dscribe.append(sine_matrix)
    now = datetime.now()
    if int(len(sine_dscribe)) == 20:
        print("matrices read:", len(sine_dscribe),", time:", (now - start))
    if int(len(sine_dscribe))%200 == 0: 
        print("matrices read:", len(sine_dscribe),", time:", (now - start))
print('Number of matrices read: ', len(sine_dscribe))
print("--- %s time taken ---" % ((datetime.now() - start)))

matrices read: 20 , time: 0:00:00.161407
matrices read: 200 , time: 0:00:00.741142
matrices read: 400 , time: 0:00:01.365780
matrices read: 600 , time: 0:00:02.063516
matrices read: 800 , time: 0:00:02.767657
matrices read: 1000 , time: 0:00:03.519281
matrices read: 1200 , time: 0:00:04.199126
matrices read: 1400 , time: 0:00:04.833629
matrices read: 1600 , time: 0:00:05.478799
matrices read: 1800 , time: 0:00:06.381453
matrices read: 2000 , time: 0:00:07.140995
matrices read: 2200 , time: 0:00:07.781644
matrices read: 2400 , time: 0:00:08.407009
matrices read: 2600 , time: 0:00:09.034300
matrices read: 2800 , time: 0:00:09.655773
matrices read: 3000 , time: 0:00:10.288895
matrices read: 3200 , time: 0:00:10.906562
matrices read: 3400 , time: 0:00:11.529085
matrices read: 3600 , time: 0:00:12.149465
matrices read: 3800 , time: 0:00:12.763192
matrices read: 4000 , time: 0:00:13.379418
Number of matrices read:  4023
--- 0:00:13.449720 time taken ---


In [92]:
#1 example descriptor, only eigenvalue
print(sine_dscribe[0])
print(sine_dscribe[0].shape)

[1582.07639598+0.00000000e+00j  844.56081641+0.00000000e+00j
  844.56081641+0.00000000e+00j  844.56081641+0.00000000e+00j
  844.56081641+0.00000000e+00j  750.30393113+0.00000000e+00j
  750.30393113+0.00000000e+00j  750.30393113+0.00000000e+00j
  750.30393113+0.00000000e+00j  580.5272067 +0.00000000e+00j
  432.2363368 +0.00000000e+00j  414.97922122+0.00000000e+00j
  414.97922122+0.00000000e+00j  414.97922122+0.00000000e+00j
  414.97922122+0.00000000e+00j  390.1105857 +0.00000000e+00j
  359.1595625 +0.00000000e+00j  359.1595625 +0.00000000e+00j
  359.1595625 +0.00000000e+00j  359.1595625 +0.00000000e+00j
  357.46335109+0.00000000e+00j  357.46335109+0.00000000e+00j
  357.46335109+0.00000000e+00j  357.46335109+0.00000000e+00j
  355.52120908+0.00000000e+00j  355.52120908+0.00000000e+00j
  355.52120908+0.00000000e+00j  355.52120908+0.00000000e+00j
  335.00356185+0.00000000e+00j  335.00356185+0.00000000e+00j
  335.00356185+0.00000000e+00j  335.00356185+0.00000000e+00j
  334.52507445+0.0000000

In [93]:
#descriptor of a full matrix in order defined by the atoms
sine = SineMatrix(
    n_atoms_max=54,
    #permutation="eigenspectrum",
    sparse=False,
    flatten=False
)
full = sine.create(ase_structures[0])
print(full)
print(full.shape)

[[8.33267490e+02 5.98196934e+01 3.45369161e+01 ... 3.33017558e+00
  4.70957947e+00 3.33017558e+00]
 [5.98196934e+01 8.33267490e+02 3.45369161e+01 ... 3.33017558e+00
  4.70957947e+00 4.70957947e+00]
 [3.45369161e+01 3.45369161e+01 8.33267490e+02 ... 4.70957947e+00
  4.70957947e+00 3.33017558e+00]
 ...
 [3.33017558e+00 3.33017558e+00 4.70957947e+00 ... 6.98330508e+00
  1.11234967e+00 1.11234967e+00]
 [4.70957947e+00 4.70957947e+00 4.70957947e+00 ... 1.11234967e+00
  6.98330508e+00 6.42215382e-01]
 [3.33017558e+00 4.70957947e+00 3.33017558e+00 ... 1.11234967e+00
  6.42215382e-01 6.98330508e+00]]
(54, 54)


## Coulomb Matrix from Matminer

In [52]:
cm_mm.fit(structures)


CoulombMatrix()

In [54]:
#CM using Matminer, this uses Pymatgen structures

#matminer descriptors
from matminer.featurizers import structure as sf

cm_matminer=[]
cm_mm = sf.CoulombMatrix(flatten=True)
start = datetime.now()
matminer_matrix = cm_mm.fit(structures)
for i,struct in enumerate(structures):
    
    featurized_structure = matminer_matrix.featurize(struct)
    cm_matminer.append(featurized_structure)
    now = datetime.now()
    if len(cm_matminer)%200 == 0: 
        print("matrices read:", len(cm_matminer),", time:", (now - start))
    
print('Number of matrices read: ', len(cm_matminer))
print("--- %s time taken ---" % ((datetime.now() - start)))

/opt/anaconda3/envs/MLenv/lib/python3.9/site-packages/matminer/featurizers/structure/matrix.py:100: ComplexWarning: Casting complex values to real discards the imaginary part
  zeros[: len(eigs)] = eigs


matrices read: 200 , time: 0:00:17.899286
matrices read: 400 , time: 0:00:35.710224
matrices read: 600 , time: 0:00:53.512635
matrices read: 800 , time: 0:01:11.467170
matrices read: 1000 , time: 0:01:29.488261
matrices read: 1200 , time: 0:01:47.401966
matrices read: 1400 , time: 0:02:05.454721
matrices read: 1600 , time: 0:02:23.581921
matrices read: 1800 , time: 0:02:41.435521
matrices read: 2000 , time: 0:02:59.449398
matrices read: 2200 , time: 0:03:17.432026
matrices read: 2400 , time: 0:03:35.661356
matrices read: 2600 , time: 0:03:53.853999
matrices read: 2800 , time: 0:04:11.956088
matrices read: 3000 , time: 0:04:30.254314
matrices read: 3200 , time: 0:04:48.307846
matrices read: 3400 , time: 0:05:07.172887
matrices read: 3600 , time: 0:05:25.598728
matrices read: 3800 , time: 0:05:43.825729


KeyboardInterrupt: 

In [80]:
cm_mm = sf.CoulombMatrix(flatten=True)
matminer_matrix = cm_mm.fit(structures)
cm_matminer_1 = matminer_matrix.featurize(structures[0])


In [81]:
print(cm_matminer_1)
print(cm_matminer_1.shape)

[1971.17128361  594.85348769  812.40237535  812.40237535  400.29414904
  702.13917465  702.13917465  366.01493461  812.40237535  812.40237535
  389.74361918  389.74361918    9.62967336  812.40237535  812.40237535
  371.27587096  371.27587096  389.74361918  389.74361918  318.20798292
  389.74361918  371.27587096  318.20798292  318.5145681   318.5145681
  318.5145681   389.74361918  371.27587096  371.27587096  371.27587096
  318.5145681   318.40672195    6.46328633    4.53258333    4.53258333
    4.53258333    4.53258333    5.11735365    5.11735365    5.11735365
    5.11735365    5.11735365    5.11735365    6.45907173    6.45907173
    6.45907173    6.45907173    6.45907173    6.45907173  318.40672195
  318.40672195  318.40672195  318.40672195  318.40672195]
(54,)


In [88]:
cm_ds = CoulombMatrix(n_atoms_max=54,permutation='eigenspectrum')
cm_1 = cm_ds.create(ase_structures[0])
print(cm_1)
print(cm_1.shape)

[2679.77619447 1133.67577892  913.07374493  841.08011951  712.6289803
  708.830714    692.25295309  656.98047438  655.93789699  611.12050477
  467.43255314  434.36093126  405.95650918  373.37691024  360.90885612
  360.73370676  330.59963438  323.17143181  321.91922861  319.70671666
  317.12026943  310.00661992  304.11487923  301.61723033  299.21583906
  298.12478161  296.60093965  292.83209275  292.52858515  292.14396424
  291.40954568  283.9316412   283.8511157   283.12161825  282.57751345
  280.79923438   12.8666636     7.39275912    7.03464999    5.57408163
    5.28041001    4.54978872    4.52989384    4.34655433    4.22686951
    4.14798147    3.80471049    3.74741204    3.72709043    3.52626779
    3.47022584    3.37089434    3.33626234    3.28758742]
(54,)


# <a id='ml'>Machine learning</a>

## Data normalisation

### MinMaxScaler

In [15]:
from sklearn.preprocessing import StandardScaler , MinMaxScaler
from sklearn.model_selection import train_test_split


#X_train, X_test, y_train, y_test = train_test_split(descriptor, energies, random_state=1, test_size = 0.5)
X_train = np.array([np.arange(10)]*2)
scaler = MinMaxScaler()  
#scaler.fit(X_train)  
#X_train = scaler.transform(X_train)  
#X_test = scaler.transform(X_test) 

In [70]:
print(X_train)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


### StandardScaler

In [69]:
from sklearn.preprocessing import StandardScaler , MinMaxScaler 

#X_train, X_test, y_train, y_test = train_test_split(descriptor, energies, random_state=1, test_size = 0.5)

scaler = StandardScaler()  
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
#X_test = scaler.transform(X_test)  

### Error analysis

In [46]:
def r2(real,pred):
    r2 = r2_score(real, pred)
    return(r2)

def mae(real,pred):
    mae = mean_absolute_error(real, pred)*1000
    return(mae)

#maximum error
def maxer(real,pred):
    maxer = max_error(real, pred)*1000
    return(maxer)

def errorgraph(real,pred):
    mae(real, pred)
    r2(real, pred)
    maxer(real,pred)
    
    plt.figure()
    plt.plot(real, pred, marker='o')
    plt.ylabel("Predicted values")
    plt.xlabel("Calculated values")
    
    vmin=min(min(real),min(pred))
    vmax=max(max(real),max(pred))
    line=np.linspace(vmin,vmax)
    plt.plot(line,line)
    
    
    plt.text(xtags,ytags[0], 'mean absolute error: %s meV' %mae)
    plt.text(xtags,ytags[2], '$R^2$ values:', r2)
    plt.text(xtags,ytags[3], 'max error: %s meV' %maxer)
    
    plt.show()
    plt.close()

### Linear Regression

In [ ]:
#linear regression
from sklearn.linear_model import LinearRegression
model = LinearRegression()

model.fit(Xtrain, ytrain)
ypred_LR = model.predict(Xtest)

errorgraph(ytest, ypred_LR)

### Gradient Boosting Decision Tree

In [72]:
#setting the parameters for GBDT 
from sklearn.ensemble import GradientBoostingRegressor

params = {
          'n_estimators': 1000, #number of boosting stages to perform, the larger the better
#          'max_depth': 4,  #limitation for the number of nodes in each tree
#          'min_samples_split': 5, #the minimum number of samples required to split a node
#          'learning_rate': 0.01,
          'loss': 'absolute_error'} #loss function that is optimised
model = GradientBoostingRegressor(**params)

In [ ]:
model.fit(Xtrain, ytrain)
ypred_GBDT = model.predict(Xtest)

plt.figure(figsize = (6, 4))
plt.scatter(ytest, ypred_GBDT)

errorgraph(ytest, ypred_GBDT)

# <a id='protocol'>Protocol</a>

# <a id='symmetry'>Symmetry analysis</a>

Selected structures:
- 8 - 4008
- 6 - 0
- 4 - 25
- 3 - 3291
- 2 - 2278
- 1 - 1829

In [17]:
selected_structures = [4008, 0, 25, 3291, 2278, 1829]
